In [1]:
#Importing required Libraries
import requests
from bs4 import BeautifulSoup

In [7]:
##Creating the object to hold to page source content
page= requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168#.XmHyzW5uIcB")
print(page)

##Creating an object to store the page source in proper html format
soup= BeautifulSoup(page.content, 'html.parser')
#print(soup)

##Storing page source belonging to store the page source in proper html format
seven_day= soup.find(id="seven-day-forecast")
#print(seven_day)

##Finding all instances of a particular class tag within the id and storing it to object
forecast_items = seven_day.find_all(class_="tombstone-container")
#print(forecast_items)

tonight= forecast_items[0]
print(tonight.prettify())

<Response [200]>
<div class="tombstone-container">
 <p class="period-name">
  Overnight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Overnight: Clear, with a low around 56. West southwest wind around 5 mph. " class="forecast-icon" src="newimages/medium/nskc.png" title="Overnight: Clear, with a low around 56. West southwest wind around 5 mph. "/>
 </p>
 <p class="short-desc">
  Clear
 </p>
 <p class="temp temp-low">
  Low: 56 °F
 </p>
</div>


# Creating Objects out of Unstructured Data

In [9]:
##Extracting the values of sub tags
period = tonight.find (class_="period-name").get_text()
short_desc= tonight.find(class_="short-desc").get_text()
temp = tonight.find (class_="temp temp-low").get_text()

print(period)
print(short_desc)
print(temp)

Overnight
Clear
Low: 56 °F


In [10]:
##Extracting the title part of the img tag
img=tonight.find("img")
desc=img['title']
print(desc)

Overnight: Clear, with a low around 56. West southwest wind around 5 mph. 


In [11]:
##Extracting all values for the period tag from the entire object
period_tags=seven_day.select(".tombstone-container .period-name")
periods=[pt.get_text() for pt in period_tags]
print(periods)

['Overnight', 'Monday', 'MondayNight', 'Tuesday', 'TuesdayNight', 'Wednesday', 'WednesdayNight', 'Thursday', 'ThursdayNight']


In [12]:
##Similarly extracting the other tags also from the entire object
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container  .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print()
print(temps)
print()
print(descs)

['Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Mostly Clear']

['Low: 56 °F', 'High: 73 °F', 'Low: 56 °F', 'High: 71 °F', 'Low: 55 °F', 'High: 69 °F', 'Low: 56 °F', 'High: 68 °F', 'Low: 56 °F']

['Overnight: Clear, with a low around 56. West southwest wind around 5 mph. ', 'Monday: Sunny, with a high near 73. Light and variable wind becoming west 10 to 15 mph in the afternoon. Winds could gust as high as 20 mph. ', 'Monday Night: Mostly clear, with a low around 56. West southwest wind 11 to 16 mph decreasing to 5 to 10 mph in the evening. Winds could gust as high as 21 mph. ', 'Tuesday: Sunny, with a high near 71. Light southwest wind becoming west southwest 9 to 14 mph in the afternoon. ', 'Tuesday Night: Mostly clear, with a low around 55. West southwest wind 9 to 11 mph. ', 'Wednesday: Sunny, with a high near 69.', 'Wednesday Night: Mostly clear, with a low around 56.', 'Thursday: Mostly sunny, with a high near 68.', 'Thursday N

# Creating Dataframe from Raw Data

In [14]:
###Combining all tags and creating a dataframe out of it
import pandas as pd
weather = pd.DataFrame({"period":periods, "short_desc":short_descs, "temp":temps,
                        "desc":descs
                    
})
weather

,period,short_desc,temp,desc
0,Overnight,Clear,Low: 56 °F,"Overnight: Clear, with a low around 56. West s..."
1,Monday,Sunny,High: 73 °F,"Monday: Sunny, with a high near 73. Light and ..."
2,MondayNight,Mostly Clear,Low: 56 °F,"Monday Night: Mostly clear, with a low around ..."
3,Tuesday,Sunny,High: 71 °F,"Tuesday: Sunny, with a high near 71. Light sou..."
4,TuesdayNight,Mostly Clear,Low: 55 °F,"Tuesday Night: Mostly clear, with a low around..."
5,Wednesday,Sunny,High: 69 °F,"Wednesday: Sunny, with a high near 69."
6,WednesdayNight,Mostly Clear,Low: 56 °F,"Wednesday Night: Mostly clear, with a low arou..."
7,Thursday,Mostly Sunny,High: 68 °F,"Thursday: Mostly sunny, with a high near 68."
8,ThursdayNight,Mostly Clear,Low: 56 °F,"Thursday Night: Mostly clear, with a low aroun..."


# Splitting the Values in "desc" Variable

In [15]:
###"desc" has the text that contains the values of "period" also
###Trying to split the "desc" string such that the "period" part is removed
new= weather["desc"].str.split(":",expand=True)
print(new)

                 0                                                  1
0        Overnight   Clear, with a low around 56. West southwest w...
1           Monday   Sunny, with a high near 73. Light and variabl...
2     Monday Night   Mostly clear, with a low around 56. West sout...
3          Tuesday   Sunny, with a high near 71. Light southwest w...
4    Tuesday Night   Mostly clear, with a low around 55. West sout...
5        Wednesday                        Sunny, with a high near 69.
6  Wednesday Night                Mostly clear, with a low around 56.
7         Thursday                 Mostly sunny, with a high near 68.
8   Thursday Night                Mostly clear, with a low around 56.


In [16]:
##Fetching the other part and replacing the desc variable
weather["desc"]=new[1]
weather

,period,short_desc,temp,desc
0,Overnight,Clear,Low: 56 °F,"Clear, with a low around 56. West southwest w..."
1,Monday,Sunny,High: 73 °F,"Sunny, with a high near 73. Light and variabl..."
2,MondayNight,Mostly Clear,Low: 56 °F,"Mostly clear, with a low around 56. West sout..."
3,Tuesday,Sunny,High: 71 °F,"Sunny, with a high near 71. Light southwest w..."
4,TuesdayNight,Mostly Clear,Low: 55 °F,"Mostly clear, with a low around 55. West sout..."
5,Wednesday,Sunny,High: 69 °F,"Sunny, with a high near 69."
6,WednesdayNight,Mostly Clear,Low: 56 °F,"Mostly clear, with a low around 56."
7,Thursday,Mostly Sunny,High: 68 °F,"Mostly sunny, with a high near 68."
8,ThursdayNight,Mostly Clear,Low: 56 °F,"Mostly clear, with a low around 56."


In [17]:
weather.to_excel("Weather_details.xlsx",header=True)

In [18]:
import cv2